In [1]:
data <- read.table(file = "T5_5_FBEETLES.DAT")
data <- data[,-1]
colnames(data) = c("Group", "y1", "y2", "y3", "y4")

Haltica_Oleracea <- as.matrix(data[c(data[, 1] == 1), 1:5])
Haltica_Oleracea <- Haltica_Oleracea[,-1]
Haltica_Carduorum <- as.matrix(data[c(data[, 1] == 2), 1:5])
Haltica_Carduorum <- Haltica_Carduorum[,-1]



**a)** Test $H_0: \mu_1 = \mu_2$ using $T^2$. State your conclusion.

We have two samples:  
  
$\text{Haltica Oleracea: } \textbf{y}_{1,1}, \textbf{y}_{1,2}, ... , \textbf{y}_{1,19} \sim N_4( \mu_1, \Sigma_1)$  
$\text{Haltica Carduorum: } \textbf{y}_{2,1}, \textbf{y}_{2,2}, ... , \textbf{y}_{2,20} \sim N_4( \mu_2, \Sigma_2)$  
  
With null and alternative hypothesis:  
 
$H_0: \mu_1 = \mu_2$ and $H_1: \mu_1 \neq \mu_2$  
  
and test statistic:
$$T^2 = \frac{n_1n_2}{n_1 + n_2} (\bar{\textbf{y}}_1 - \bar{\textbf{y}}_2)' S_{pl}^{-1} (\bar{\textbf{y}}_1 - \bar{\textbf{y}}_2) \sim T^2_{p,n_1 + n_2 -2}$$  
where
$$ S_{pl} = \frac{(n_1 - 1)S_1 + (n_2 -1)S_2}{n_1+n_2 -2}$$
Calculating the oberserved test statistic we get:


In [2]:
# Preliminary Calculations
n1 = 19
n2 = 20
J <- function(n) {
    matrix <- matrix(nrow = n, ncol = n)
    matrix[,] = 1
    return(matrix)}

mu1 <- matrix(nrow = 4, ncol = 1)
for (i in 1:4) {mu1[i,] = sum(Haltica_Oleracea[,i] / n1)}

mu2 <- matrix(nrow = 4, ncol = 1)
for (i in 1:4) {mu2[i,] = sum(Haltica_Carduorum[,i] / n2)}

S1 <- (t(Haltica_Oleracea) %*% (diag(n1) - (1 / n1) * J(n1)) %*% Haltica_Oleracea) / (n1 -1)
S2 <- (t(Haltica_Carduorum) %*% (diag(n2) - (1 / n2) * J(n2)) %*% Haltica_Carduorum) / (n2 -1)
Spl <- ((n1 - 1) * S1 + (n2 - 1) * S2) / (n1 + n2 - 2)

# Test Statistic
T2_obs = (n1 * n2 / (n1 + n2)) * t(mu1 - mu2) %*% solve(Spl) %*% (mu1 - mu2)
T2_obs

133.4873


Calculate the p-value by converting it into an F-statistic like so,

In [3]:
pf((n1 + n2 - 4 -1) / ((n1 + n2 - 2)* 4) * T2_obs, 4, n1+n2-4-1, lower.tail = FALSE)

7.521799e-11


We reject the null hypothesis in favour of the alternative.

**b)** Carry out a t-test on each variable and state your conclusion.

For each of the 4 variables being tested we have:  
  
$\text{Haltica Oleracea: } y_{1,1}, y_{1,2}, ... , y_{1,19} \sim N( \mu_1, \sigma^2_1)$  
$\text{Haltica Carduorum: } y_{2,1}, y_{2,2}, ... , y_{2,20} \sim N( \mu_2, \sigma^2_2)$  

With null and alternative hypothesis:  
 
$H_0: \mu_1 = \mu_2$ and $H_1: \mu_1 \neq \mu_2$  
  
and test statistic: 
$$t = \frac{\bar y_1 - \bar y_2}{S_{pl} \sqrt{1/n_1 + 1/n_2}} \sim t_{n_1 + n_2 -2}$$
where
$$S^2_{pl} = \frac{(n_1 - 1)s_1^2 + (n_2-1)s_2^2}{n_1+n_2-2}$$
Calculating the observed test statistics and p-values for each of the 4 variables, we get:

In [4]:
S2pl <- matrix(nrow = 4, ncol = 1)
for ( i in 1:4) {S2pl[i,] = ( (n1-1) * S1[i,i] + (n2-1) * S2[i,i] ) / (n1 + n2 -2)}
 
t <- matrix(nrow = 4, ncol = 1)
for (i in 1:4) {t[i,] = round((mu1[i,] - mu2[i,]) / sqrt(S2pl[i,] * (1/n1 + 1/n2)),4)}

p <- matrix(nrow = 4, ncol =1)
for (i in 1:4) { p[i,] = 2 * pt(abs(t[i,]), n1 + n2 - 2, lower.tail = FALSE) }

table <- cbind(t,p)
colnames(table) <- c("t-statistic", "p-value")
rownames(table) <- c("y1","y2","y3","y4")
table

,t-statistic,p-value
y1,3.8879,4.049717e-04
y2,-3.8652,4.326723e-04
y3,-5.6911,1.645551e-06
y4,-5.0426,1.236524e-05


We reject the null hypothesis for each variable when tested individually.

**c)** Compute vector $\textbf{a} = \textbf{S}_{pl}^{-1}(\bar{\textbf{y}}_1 - \bar{\textbf{y}}_2)$. Define random variable $ x = \textbf{a}' \textbf{y}$, with $\textbf{y} = (y_1,y_2,y_3,y_4)'$. Let $\mu_x = E(x)$. Is $\mu_x$ different for the two species? Use a test to state your conclusion.

First calculate our vector $\textbf{a}$:

In [5]:
a <- solve(Spl) %*% (mu1 - mu2)
round(a,4)

y1,0.3452
y2,-0.1304
y3,-0.1064
y4,-0.1434


In [6]:
# Double Check
((t(a)%*%mu1 - t(a)%*%mu2) / sqrt( (n1+n2)/(n1*n2)  * t(a) %*% Spl %*% a))^2

133.4873


$E(\textbf{x}_1) = \textbf{a}'E(\bar{\textbf{y}}_1) = -8.9554$

In [7]:
round(t(a) %*% mu1,4)

-8.9554


$E(\textbf{x}_2) = \textbf{a}'E(\bar{\textbf{y}}_2) = -22.6554$

In [8]:
round(t(a) %*% mu2,4)

-22.6554


For the 2 transformed variables being tested we have:  
  
$x_1: x_{1,1}, x_{1,2}, ... , x_{1,19} \sim N( \textbf{a}'\mu_1, \textbf{a}'\textbf{S}_1 \textbf{a})$  
$x_2: x_{2,1}, x_{2,2}, ... , x_{2,19} \sim N( \textbf{a}'\mu_2, \textbf{a}'\textbf{S}_2 \textbf{a})$  

With null and alternative hypothesis:  
 
$H_0: \mu_{x_1} = \mu_{x_2}$ and $H_1: \mu_{x_1} \neq \mu_{x_2}$  
  
and test statistic: 
$$t = \frac{\bar x_1 - \bar x_2}{S_{pl} \sqrt{1/n_1 + 1/n_2}} \sim t_{n_1 + n_2 -2}$$
where
$$S^2_{pl} = \frac{(n_1 - 1)\textbf{a}'\textbf{S}_1\textbf{a} + (n_2-1)\textbf{a}' \textbf{S}_2 \textbf{a}}{n_1+n_2-2}$$
Calculating the observed test statistic we get:

In [9]:
S2plx = ( (n1-1) * t(a) %*% S1 %*% a + (n2-1) * t(a) %*% S2 %*% a ) / (n1 + n2 -2)
txobs = ( t(a) %*% mu1 - t(a) %*% mu2 ) / sqrt(S2plx * (1/n1 + 1/n2))
txobs

11.55367


which gives p-value:

In [10]:
px = 2 * pt(abs(txobs), n1 + n2 - 2, lower.tail = FALSE)
px

7.770049e-14


We reject the null hypothesis that $\mu_x$ is the same for the two species.